<a href="https://colab.research.google.com/github/kunwarsharik/SharikProjects/blob/master/LSTM_without_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import copy, numpy as np
np.random.seed(0)

In [0]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output
def sigmoid_output_to_derivative(output):
    return output*(1-output)#diff of sigmoid

In [0]:
int2binary = {}#look up table int to binary representation of number, we'll do math on binary rep
binary_dim = 8
largest_number = pow(2,binary_dim)

In [0]:
binary=np.unpackbits(np.array([range(largest_number)],dtype=np.uint8).T,axis=1)

In [0]:
for i in range(largest_number):
    int2binary[i] = binary[i]

In [0]:
alpha = 0.1#lR
input_dim = 2
hidden_dim = 16
output_dim = 1

In [0]:
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) -1
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) -1

synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [8]:
for j in range(10000):
    a_int = np.random.randint(largest_number/2)
    a = int2binary[a_int]
    b_int = np.random.randint(largest_number/2)
    b = int2binary[b_int]
    c_int = a_int + b_int
    c = int2binary[c_int]
    d = np.zeros_like(c)
    overallError = 0
    layer_2_deltas=[]#output layers
    layer_1_values=[]#hidden layer values
    layer_1_values.append(np.zeros(hidden_dim))
    for position in range(binary_dim):
        X = np.array([[a[binary_dim-position-1],b[binary_dim-position-1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T
        #prop input to current hidden, prop prev hidden to currt hidden add both and do sigmoid
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))#hidden layer to output
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))        
        overallError += np.abs(layer_2_error[0])
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        layer_1_values.append(copy.deepcopy(layer_1))
    future_layer_1_delta = np.zeros(hidden_dim)
    for position in range(binary_dim):#back prop from last timestep to first
        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]#currnt hidden layer
        prev_layer_1 = layer_1_values[-position-2]#prev hidden layer
        layer_2_delta = layer_2_deltas[-position-1]#current output error
        #we have curnt output layer error and hidden layer error from future layer, cuurent hidn layer error can be calc
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)
        #we calc wt updates but actual update will be done when complete back prop done
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        future_layer_1_delta = layer_1_delta
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    if(j % 1000 == 0):
        print("error:"+str(overallError)+"pred:"+str(d)+"true")
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print (str(a_int)+"+"+str(b_int)+"="+str(out))
        print("----------------------------------------")

error:[3.45638663]pred:[0 0 0 0 0 0 0 1]true
9+60=1
----------------------------------------
error:[3.63389116]pred:[1 1 1 1 1 1 1 1]true
28+35=255
----------------------------------------
error:[3.91366595]pred:[0 1 0 0 1 0 0 0]true
116+44=72
----------------------------------------
error:[3.72191702]pred:[1 1 0 1 1 1 1 1]true
4+73=223
----------------------------------------
error:[3.5852713]pred:[0 0 0 0 1 0 0 0]true
71+11=8
----------------------------------------
error:[2.53352328]pred:[1 0 1 0 0 0 1 0]true
81+113=162
----------------------------------------
error:[0.57691441]pred:[0 1 0 1 0 0 0 1]true
81+0=81
----------------------------------------
error:[1.42589952]pred:[1 0 0 0 0 0 0 1]true
4+125=129
----------------------------------------
error:[0.47477457]pred:[0 0 1 1 1 0 0 0]true
39+17=56
----------------------------------------
error:[0.21595037]pred:[0 0 0 0 1 1 1 0]true
11+3=14
----------------------------------------
